<a href="https://colab.research.google.com/github/shiva-samy/algotrading-zerodha/blob/main/Timetable_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Automatic Timetable Generator using Genetic algorithm

**Problem Statement:**

Provide solution to develop an automated Timetable Generation System for college that optimally assigns classes to students and teachers while adhering to various constraints and preferences.

**Guidelines:**
* Inputs:
 * For each class, List of subjects along with number of periods per week for each
subject
 * Subject allotment - Handling faculty for each subject
 * Elective Subjects (Parallel allotment possibility)
 * Combined classes if any
 * Workload of each faculty
 * Classrooms with capacity
 * Laboratories with capacity
 * TWM, Seminar etc.. may be considered as one subject and included in the workload
* Outputs:
 * Class Timetable
 * Faculty Timetable
 * Classroom wise workload
 * Lab Timetable
* Constraints and Requirements to consider:Staff Schedule:
 * Ensure that teachers do not have consecutive teaching hours. Maximum 3 teaching
periods per day.
 * Lab classes require at least 2 teachers.
 * Teacher should have class on all days.
* Room and Lab Availability:
 * Ensure the availability of classrooms and labs.
 * Maximize the utilization of classrooms and labs.
 * Only one lab class should be scheduled per day. Unavoidable situation you can schedule 2.
* Class Timing:
 * 5 days per week and 7 (4+3) periods per day. In some case you can have 4 periods in the afternoon while scheduling lab classes.

In [ ]:
import pandas as pd
import numpy as np
import random
from collections import OrderedDict

In the above code, we are importing the libraries necessary for running the timetable scheduling algorithm.

In [ ]:
df = pd.read_csv("/content/subjects.csv")
print(df)

  Subject  No.of Hrs Type  Combined Class Faculties
0  19z401          4  Lec           False    S1, S5
1  19z402          3  Lec           False    S2, S4
2  19z403          3  Lec           False    S1, S3
3  19z404          4  Lec            True    S2, S4
4  19z405          3  Lec           False    S3, S5
5  19z411          4  Lab           False    S1, S6
6  19z412          2  Lab           False    S3, S4


We see that the imported csv file contains the columns "Subject", "No.of Hrs", "Type", "Combined Class" (boolean column), "Faculties" (we consider each course is assigned 2 faculties)

In [ ]:
slots = ['08:30', '09:20', '10:30', '11:20', '01:40', '02:30', '03:30', '04:20']

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

# Generating key-value pairs for subject
subject = {}
for index, row in df.iterrows():
    key = row['Subject']
    value = [row['No.of Hrs'], row['Type']]
    subject[key] = value

print("Subjects:")
print(subject)

# Generating key-value pairs for staff
staff = {}
for index, row in df.iterrows():
    faculties = row['Faculties'].split(', ')
    for faculty in faculties:
        if faculty in staff:
            staff[faculty].append(row['Subject'])
        else:
            staff[faculty] = [row['Subject']]

staff=dict(sorted(staff.items()))
print("Staff:")
print(staff)


Subjects:
{'19z401': [4, 'Lec'], '19z402': [3, 'Lec'], '19z403': [3, 'Lec'], '19z404': [4, 'Lec'], '19z405': [3, 'Lec'], '19z411': [4, 'Lab'], '19z412': [2, 'Lab']}
Staff:
{'S1': ['19z401', '19z403', '19z411'], 'S2': ['19z402', '19z404'], 'S3': ['19z403', '19z405', '19z412'], 'S4': ['19z402', '19z404', '19z412'], 'S5': ['19z401', '19z405'], 'S6': ['19z411']}


Now we have created the required data (slots, days, subject, staff) in the format the genetic algorithm supports.

In [57]:
# Function to generate a random valid timetable

def allocate_class(timetable, faculty_schedule, day, slot, sub, subject_count):
    # Check if the slot is available in both timetable and faculty schedule
    if np.isnan(timetable.loc[day, slot]) and np.isnan(faculty_schedule.loc[day, slot]):
        timetable.loc[day, slot] = sub
        faculty_schedule.loc[day, slot] = sub
        subject_count[sub] += 1
        return True
    else:
        return False


def generate_valid_timetable():
    timetable = pd.DataFrame(index=days, columns=slots, data=None)
    subject_count = {sub: 0 for sub in df["Subject"]}
    daily_workload = {day: 0 for day in days}

    consecutive_hours = {sub: 0 for sub in df[df["Type"] == "Lec"]["Subject"]}
    weekly_hours = {day: 0 for day in days}

    lab_classes = df[df["Type"] == "Lab"]
    lec_classes = df[df["Type"] == "Lec"]

    # Define priority slots for lab classes with 2 hours duration
    lab_priority_slots = [['08:30', '09:20'], ['10:30', '11:20'], ['01:40', '02:30']]

    # Iterate through lab classes first
    for idx, row in lab_classes.iterrows():
        sub = row["Subject"]
        hours = row["No.of Hrs"]
        classtype = row["Type"]

        if classtype == "Lab":
            if hours == 4:
                # For Lab classes of 4 hours, select a random day and a valid slot range
                day = random.choice(days)
                valid_slots = [['08:30', '09:20', '10:30', '11:20'], ['01:40', '02:30', '03:30', '04:20']]
                slot_range = random.choice(valid_slots)

                # Check if there are enough slots available for lab class
                if all(pd.isna(timetable.loc[day,slot]) for slot in slot_range):
                    # Assign the lab class to the selected slots
                    for slot in slot_range:
                        timetable.loc[day, slot] = sub
                        subject_count[sub] += 1
                        daily_workload[day] += 1
            else:  # For Lab classes of 2 hours
                # Select a random day and a valid slot range from priority slots
                day = random.choice(days)
                slot_range = random.choice(lab_priority_slots)

                # Check if there are enough slots available for lab class
                if all(pd.isna(timetable.loc[day,slot]) for slot in slot_range):
                    # Assign the lab class to the selected slots
                    for slot in slot_range:
                        timetable.loc[day, slot] = sub
                        subject_count[sub] += 1
                        daily_workload[day] += 1

    # For Lecture classes
    for idx, row in lec_classes.iterrows():
        sub = row["Subject"]
        hours = row["No.of Hrs"]
        classtype = row["Type"]

        for _ in range(hours):
            # Prioritize slots from 8:30 to 2:30 for lecture classes
            prioritized_slots = [slot for slot in slots if slot in ['09:20', '10:30', '11:20', '01:40', '02:30']]
            day, slot = None, None
            for ps in prioritized_slots:
                day = random.choice(days)
                if pd.isna(timetable.loc[day, ps]):
                    slot = ps
                    break
            if day is None or slot is None:
                # If the prioritized slots are filled, choose randomly from all slots
                day = random.choice(days)
                slot = random.choice(slots)
                while not pd.isna(timetable.loc[day, slot]):
                    day = random.choice(days)
                    slot = random.choice(slots)
            timetable.loc[day, slot] = sub
            subject_count[sub] += 1
            daily_workload[day] += 1

    # Check if all subjects have been allocated the correct number of hours
    for sub, count in subject_count.items():
        if count != df[df["Subject"] == sub]["No.of Hrs"].values[0]:
            # If the count doesn't match the number of hours, regenerate timetable
            return generate_valid_timetable()

    # Check if the workload is distributed evenly across days
    max_workload = max(daily_workload.values())
    min_workload = min(daily_workload.values())
    workload_difference = max_workload - min_workload
    if workload_difference > 1:  # Adjust as needed based on your workload balancing preference
        # If workload difference is greater than 1, regenerate timetable
        return generate_valid_timetable()

    print(timetable)
    print(subject_count)
    return timetable

            # for _ in range(hours):
            #     day = random.choice(days)
            #     slot = random.choice(slots)

            #     #Repeatedly check for empty slots
            #     while(timetable.loc[day,slot]!=''):
            #       day = random.choice(days)
            #       slot = random.choice(slots)

            #     # Check if the slot is available
            #     if timetable.loc[day, slot] == '':
            #         timetable.loc[day, slot] = sub
            #         subject_count[sub] += 1

            #     #     # Reset consecutive lecture count
            #         consecutive_lecture_count = 0
            #     else:
            #         # Increment consecutive lecture count
            #         consecutive_lecture_count += 1

            #         # If consecutive lecture count exceeds the maximum, break and regenerate timetable
            #         if consecutive_lecture_count > 2:
            #             return generate_valid_timetable()

def calculate_fitness(timetable):
    days = timetable.index
    slots = timetable.columns

    # Calculate additional penalty points for constraints violation
    penalty_points = 0
    for day in days:
        consecutive_hours = 0
        prev_subject = None
        prev_slot = None
        for slot in slots:
            if pd.isna(timetable.loc[day, slot]):
                consecutive_hours += 1
                if prev_subject == "Break" and prev_slot == "01:40" and slot == "02:30":
                    penalty_points -= 1
                if consecutive_hours > 7:
                    penalty_points += 5  # Add penalty for exceeding 7 hours
                    break  # No need to continue checking if limit is reached
            else:
                subject = timetable.loc[day, slot]

                if prev_subject is not None and subject != prev_subject:
                    if prev_slot is not None and prev_slot != "04:20" and slot != "08:30":
                        penalty_points -= 1

                consecutive_hours = 0
                prev_subject = subject
            prev_slot = slot

    # Calculate total fitness
    total_fitness = -penalty_points  # Penalize for violations
    return total_fitness

def selection(population, fitness_scores):
    # Sort indices based on fitness scores in descending order
    sorted_indices = sorted(range(len(fitness_scores)), key=lambda k: fitness_scores[k], reverse=True)

    # Select the best half of the population
    selected_indices = sorted_indices[:len(population)//2]
    selected_individuals = [population[i] for i in selected_indices]

    # If the population size is odd, randomly select one more individual
    if len(selected_individuals) % 2 != 0:
        selected_individuals.append(random.choice(population))

    return selected_individuals

def crossover(parent1, parent2):
    # Single-point crossover
    crossover_point = np.random.randint(1, min(len(parent1), len(parent2)))
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
    return offspring1, offspring2

def mutation(individual):
    individual_length = len(individual)
    # Adjust the mutation size based on the length of the individual
    mutation_size = np.random.randint(1, individual_length + 1)  # Ensure mutation_size is within bounds
    mutation_indices = random.sample(range(individual_length), mutation_size)
    mutated_individual = individual.copy()
    random.shuffle(mutation_indices)
    mutated_individual.iloc[mutation_indices] = individual.iloc[mutation_indices].sample(frac=1).values
    return mutated_individual

# Genetic algorithm main loop
def genetic_algorithm(population_size, generations):
    # Initialize population
    population = initialize_population(population_size)

    for generation in range(generations):
        # Evaluate fitness of each individual in the population
        fitness_scores = [calculate_fitness(individual) for individual in population]

        # Select individuals for crossover
        selected_parents = selection(population, fitness_scores)

        # Perform crossover to create offspring
        offspring = []
        for i in range(0, len(selected_parents), 2):
            parent1 = selected_parents[i]
            parent2 = selected_parents[i+1]
            child1, child2 = crossover(parent1, parent2)
            offspring.extend([child1, child2])

        # Perform mutation on offspring
        mutated_offspring = [mutation(individual) for individual in offspring]

        # Replace the weaker half of the old population with the stronger half of the new mutated population
        population = mutated_offspring + selected_parents[len(selected_parents)//2:]

    # Select the best individual from the final population as the solution
    best_individual = max(population, key=calculate_fitness)

    return best_individual

def initialize_population(population_size):
    population = []
    for _ in range(population_size):
        timetable = generate_valid_timetable()
        population.append(timetable)
    return population

# Call the genetic algorithm
best_timetable = genetic_algorithm(population_size=10, generations=2)
print("Best Timetable:")
print(best_timetable)



            08:30   09:20   10:30   11:20   01:40   02:30 03:30   04:20
Monday     19z412  19z412  19z403  19z403     NaN     NaN   NaN     NaN
Tuesday       NaN  19z402  19z401  19z403  19z402  19z404   NaN     NaN
Wednesday     NaN  19z402     NaN  19z401  19z405     NaN   NaN  19z405
Thursday      NaN  19z401  19z401  19z404  19z404  19z404   NaN     NaN
Friday     19z411  19z411  19z411  19z411  19z405     NaN   NaN     NaN
{'19z401': 4, '19z402': 3, '19z403': 3, '19z404': 4, '19z405': 3, '19z411': 4, '19z412': 2}
            08:30   09:20   10:30   11:20   01:40   02:30 03:30 04:20
Monday        NaN  19z403  19z401  19z403  19z402  19z404   NaN   NaN
Tuesday    19z411  19z411  19z411  19z411  19z405     NaN   NaN   NaN
Wednesday     NaN  19z401  19z402  19z403  19z405     NaN   NaN   NaN
Thursday      NaN  19z401  19z402  19z404  19z405     NaN   NaN   NaN
Friday     19z412  19z412  19z401  19z404  19z404     NaN   NaN   NaN
{'19z401': 4, '19z402': 3, '19z403': 3, '19z404': 4, '19

generate_valid_timetable() Function:

* This function generates a random valid timetable by allocating classes based on certain constraints.
* It initializes an empty timetable and dictionaries to keep track of subject counts and daily workloads.
* It prioritizes lab classes by selecting slots for 4-hour labs and 2-hour labs from pre-defined priority slots.
* For lecture classes, it prioritizes slots from 8:30 to 2:30. If those slots are filled, it randomly selects from all slots.
* After allocating classes, it checks if each subject has been assigned the correct number of hours and if the workload is evenly distributed across days.
* If any constraints are violated, it regenerates the timetable.

calculate_fitness() Function:

* This function evaluates the fitness of a timetable based on certain criteria.
* It penalizes timetables for violations such as exceeding 7 hours per day, having consecutive classes without breaks, and not filling prioritized slots.

selection() Function:

* This function selects individuals (timetables) for crossover based on their fitness scores.
* It sorts the indices of individuals based on fitness scores in descending order and selects the best half of the population.
* If the population size is odd, it randomly selects one more individual to maintain the population size.

crossover() Function:

* This function performs single-point crossover between pairs of parents to produce offspring.
* It randomly selects a crossover point and combines the genetic information of two parents to create two offspring.

mutation() Function:

* This function introduces random changes (mutations) to individual timetables to explore new solutions.
* It randomly selects a subset of genes (slots) and shuffles their values to introduce variability.

Genetic Algorithm Main Loop (genetic_algorithm() Function):

* It initializes a population of timetables.
* For a specified number of generations, it evaluates the fitness of each individual in the population, selects parents for crossover, performs crossover and mutation to create offspring, and replaces the weaker half of the old population with the stronger half of the new population.
* It prints information about each generation's best fitness and average fitness.
* Finally, it selects the best individual (timetable) from the final population as the solution.


Overall, this algorithm aims to iteratively improve the quality of timetables by evolving populations of solutions through selection, crossover, and mutation, while ensuring adherence to constraints and maximizing fitness.

We observed that for the given data, the following paramaters gave the best output

Generations: 2

Population: 10